In [1]:
import pandas as pd
from evaluate import load

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "2"

bleu = load("sacrebleu")
chrf = load("chrf")
cometm = load("comet")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-25 19:40:38.888257: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 19:40:38.953895: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 19:40:41.198015: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different n

In [2]:
!nvidia-smi

Tue Nov 25 19:40:58 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   58C    P0              73W / 300W |  66179MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
def compute_scores(preds, ref):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    bleu_score = bleu.compute(predictions=preds, references=ref)["score"]
    chrf_score = chrf.compute(predictions=preds, references=ref)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=ref, char_order=6, word_order=2,  beta=2)["score"]
    comet_score = cometm.compute(sources=src_sentences, predictions=preds, references=ref)["mean_score"]
    return bleu_score, chrf_score, chrf2_score, comet_score

In [4]:
import os
os.getcwd()

'/workspace/Approach0/thenlpresearcher/in22_conv_eng_mar_hin'

## Original

In [5]:
file_name = "original_outputs.csv"
mode = "original"

In [6]:
df = pd.read_csv(file_name)
df.head()

,src,prediction_mar,prediction_hin,gt_mar,gt_hin
0,"Mom, let's go for a movie tomorrow.","आई, उद्या चित्रपटाला जाऊया.","माँ, चलो कल एक फ़िल्म देखने चलते हैं।","आई, उद्या आपण एखादा सिनेमा बघायला जाऊया.","माँ, चलो कल एक फिल्म देखने चलते हैं।"
1,I don't have to go to school.,मला शाळेत जाण्याची गरज नाही.,मुझे स्कूल जाने की ज़रूरत नहीं है।,मला शाळेत जायचं नाहीये.,मुझे कल स्कूल नहीं जाना है।
2,It is a holiday.,ती सुट्टी असते.,यह छुट्टी है।,उद्या सुट्टी आहे.,छुट्टी है।
3,"Oh, tomorrow is the 14th of April right?","अरे, उद्या 14 एप्रिल आहे ना?","ओह, कल 14 अप्रैल है ना?","ओहो, उद्या १४ एप्रिल हो ना?","ओह, कल 14 अप्रैल है न?"
4,Your dad will also have the day off from work.,तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.,आपके पिता को भी काम से छुट्टी मिलेगी।,तुझ्या बाबांना सुद्धा उद्या सुट्टी राहील.,कल तुम्हारे पिताजी की भी छुट्टी होगी।


In [7]:
src_sentences = df['src']
ref_gem = df['gt_mar']
translations = df['prediction_mar']

In [8]:
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'original'=====
BLEU: 18.95, chrF++: 51.05, chrF2++: 47.74, COMET: 0.8117


## Only Punctuations

In [9]:
file_name = "with_outputs.csv"
mode = "with"

In [10]:
df = pd.read_csv(file_name)
df.head()

,src,prediction_mar,prediction_hin,gt_mar,gt_hin
0,"Mom, let's go for a movie tomorrow.","आई, उद्या चित्रपट बघायला जाऊ.","माँ, चलो कल एक फ़िल्म देखने चलते हैं।","आई, उद्या आपण एखादा सिनेमा बघायला जाऊया.","माँ, चलो कल एक फिल्म देखने चलते हैं।"
1,I don't have to go to school.,मला शाळेत जाण्याची गरज नाही.,मुझे स्कूल जाने की ज़रूरत नहीं है।,मला शाळेत जायचं नाहीये.,मुझे कल स्कूल नहीं जाना है।
2,It is a holiday.,ती सुट्टी असते.,यह छुट्टी है।,उद्या सुट्टी आहे.,छुट्टी है।
3,"Oh, tomorrow is the 14th of April right?","अरे, उद्या 14 एप्रिल आहे ना?","ओह, कल 14 अप्रैल है ना?","ओहो, उद्या १४ एप्रिल हो ना?","ओह, कल 14 अप्रैल है न?"
4,Your dad will also have the day off from work.,तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.,आपके पिता को भी काम से छुट्टी मिलेगी।,तुझ्या बाबांना सुद्धा उद्या सुट्टी राहील.,कल तुम्हारे पिताजी की भी छुट्टी होगी।


In [11]:
src_sentences = df['src']
ref_gem = df['gt_mar']
translations = df['prediction_mar']

In [12]:
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'with'=====
BLEU: 16.08, chrF++: 49.77, chrF2++: 46.65, COMET: 0.7920


## Without Punctuation

In [13]:
file_name = "without_outputs.csv"
mode = "without"

In [14]:
df = pd.read_csv(file_name)
df.head()

,src,prediction_mar,prediction_hin,gt_mar,gt_hin
0,"Mom, let's go for a movie tomorrow.","आई, उद्या चित्रपट बघायला जाऊ.","माँ, चलो कल एक फ़िल्म देखने चलते हैं।","आई, उद्या आपण एखादा सिनेमा बघायला जाऊया.","माँ, चलो कल एक फिल्म देखने चलते हैं।"
1,I don't have to go to school.,मला शाळेत जायची गरज नाही.,मुझे स्कूल जाने की ज़रूरत नहीं है।,मला शाळेत जायचं नाहीये.,मुझे कल स्कूल नहीं जाना है।
2,It is a holiday.,ती सुट्टी असते.,यह छुट्टी है।,उद्या सुट्टी आहे.,छुट्टी है।
3,"Oh, tomorrow is the 14th of April right?","अरे, उद्या 14 एप्रिल आहे ना?","ओह, कल 14 अप्रैल है ना?","ओहो, उद्या १४ एप्रिल हो ना?","ओह, कल 14 अप्रैल है न?"
4,Your dad will also have the day off from work.,तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.,आपके पिता को भी काम से छुट्टी मिलेगी।,तुझ्या बाबांना सुद्धा उद्या सुट्टी राहील.,कल तुम्हारे पिताजी की भी छुट्टी होगी।


In [15]:
src_sentences = df['src']
ref_gem = df['gt_mar']
translations = df['prediction_mar']

In [16]:
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'without'=====
BLEU: 16.67, chrF++: 49.68, chrF2++: 46.72, COMET: 0.7995


## Combined LE

In [17]:
file_name = "combined_2x_outputs.csv"
mode = "combined_2x"

In [18]:
df = pd.read_csv(file_name)
df.head()

,src,prediction_mar,prediction_hin,gt_mar,gt_hin
0,"Mom, let's go for a movie tomorrow.","आई, उद्या एखाद्या चित्रपटाला जाऊ.","माँ, चलो कल एक फ़िल्म देखने चलते हैं।","आई, उद्या आपण एखादा सिनेमा बघायला जाऊया.","माँ, चलो कल एक फिल्म देखने चलते हैं।"
1,I don't have to go to school.,मला शाळेत जाण्याची गरज नाही.,मुझे स्कूल जाने की ज़रूरत नहीं है।,मला शाळेत जायचं नाहीये.,मुझे कल स्कूल नहीं जाना है।
2,It is a holiday.,ती सुट्टी असते.,यह छुट्टी है।,उद्या सुट्टी आहे.,छुट्टी है।
3,"Oh, tomorrow is the 14th of April right?","अरे, उद्या 14 एप्रिल आहे ना?","ओह, कल 14 अप्रैल है ना?","ओहो, उद्या १४ एप्रिल हो ना?","ओह, कल 14 अप्रैल है न?"
4,Your dad will also have the day off from work.,तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.,आपके पिता को भी काम से छुट्टी मिलेगी।,तुझ्या बाबांना सुद्धा उद्या सुट्टी राहील.,कल तुम्हारे पिताजी की भी छुट्टी होगी।


In [19]:
src_sentences = df['src']
ref_gem = df['gt_mar']
translations = df['prediction_mar']

In [20]:
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'combined_2x'=====
BLEU: 17.93, chrF++: 50.45, chrF2++: 47.33, COMET: 0.8065


## Combined LED

In [21]:
file_name = "combined_x_outputs.csv"
mode = "combined_x"

In [22]:
df = pd.read_csv(file_name)
df.head()

,src,prediction_mar,prediction_hin,gt_mar,gt_hin
0,"Mom, let's go for a movie tomorrow.","आई, उद्या एखाद्या चित्रपटाला जाऊ.","माँ, चलो कल एक फ़िल्म देखने चलते हैं।","आई, उद्या आपण एखादा सिनेमा बघायला जाऊया.","माँ, चलो कल एक फिल्म देखने चलते हैं।"
1,I don't have to go to school.,मला शाळेत जाण्याची गरज नाही.,मुझे स्कूल जाने की ज़रूरत नहीं है।,मला शाळेत जायचं नाहीये.,मुझे कल स्कूल नहीं जाना है।
2,It is a holiday.,ती सुट्टी असते.,यह छुट्टी है।,उद्या सुट्टी आहे.,छुट्टी है।
3,"Oh, tomorrow is the 14th of April right?","अरे, उद्या 14 एप्रिल आहे ना?","ओह, कल 14 अप्रैल है ना?","ओहो, उद्या १४ एप्रिल हो ना?","ओह, कल 14 अप्रैल है न?"
4,Your dad will also have the day off from work.,तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.,आपके पिता को भी काम से छुट्टी मिलेगी।,तुझ्या बाबांना सुद्धा उद्या सुट्टी राहील.,कल तुम्हारे पिताजी की भी छुट्टी होगी।


In [23]:
src_sentences = df['src']
ref_gem = df['gt_mar']
translations = df['prediction_mar']

In [24]:
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'combined_x'=====
BLEU: 17.87, chrF++: 50.34, chrF2++: 47.26, COMET: 0.8082


## T5

In [25]:
file_name = "t5_outputs.csv"
mode = "t5"

In [26]:
df = pd.read_csv(file_name)
df.head()

,src,prediction_mar,prediction_hin,gt_mar,gt_hin
0,"Mom, let's go for a movie tomorrow.","आई, उद्या एखाद्या चित्रपटाला जाऊ.","माँ, चलो कल एक फ़िल्म देखने चलते हैं।","आई, उद्या आपण एखादा सिनेमा बघायला जाऊया.","माँ, चलो कल एक फिल्म देखने चलते हैं।"
1,I don't have to go to school.,मला शाळेत जाण्याची गरज नाही.,मुझे स्कूल जाने की ज़रूरत नहीं है।,मला शाळेत जायचं नाहीये.,मुझे कल स्कूल नहीं जाना है।
2,It is a holiday.,ती सुट्टी असते.,यह छुट्टी है।,उद्या सुट्टी आहे.,छुट्टी है।
3,"Oh, tomorrow is the 14th of April right?","अरे, उद्या 14 एप्रिल आहे ना?","ओह, कल 14 अप्रैल है ना?","ओहो, उद्या १४ एप्रिल हो ना?","ओह, कल 14 अप्रैल है न?"
4,Your dad will also have the day off from work.,तुमच्या वडिलांनाही कामावरून सुट्टी मिळेल.,आपके पिता को भी काम से छुट्टी मिलेगी।,तुझ्या बाबांना सुद्धा उद्या सुट्टी राहील.,कल तुम्हारे पिताजी की भी छुट्टी होगी।


In [27]:
src_sentences = df['src']
ref_gem = df['gt_mar']
translations = df['prediction_mar']

In [28]:
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 't5'=====
BLEU: 17.82, chrF++: 50.24, chrF2++: 47.17, COMET: 0.8077
